# Land Parcel Identification System (LPIS) prediction for Slovenia

This notebook shows the steps towards constructing machine learning model for LPIS prediction for Slovenia.

### Overview

#### Requirements
1. Downloaded and processed Sentinel data *(relevant [notebook](https://github.com/sentinel-hub/eo-learn/blob/master/examples/land-cover-map/SI_LULC_pipeline.ipynb))*
    * Sentinel-2 data download
    * cloud detection and masking
    * interpolation    
    

2. Downloaded and grouped LPIS data *(relevant [notebook](TODO))*
    * LPIS data download
    * LPIS class grouping 
    
#### Samples construction
1. Data sample construction
    * edge mask construction
    * oversampling
2. Feature calculation
    * stream feature calculation
    * elevation
    
#### Feature selection and model construction
1. Feature selection
    * FASTENER
2. Model construction
    * data normalization
    * model training
    * model testing
3. Model usage
    * prediction of LPIS on chosen region



In [ ]:
# Firstly, some necessary imports

# Jupyter notebook related
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import numpy as np
np.random.seed(42)

import matplotlib as mpl

# from eolearn.mask import EdgeExtractionTask
from edge_extraction import EdgeExtractionTask # Change once it will be in develop

from eolearn.core import EOTask, EOPatch, LinearWorkflow, FeatureType, OverwritePermission, \
    LoadTask, SaveTask, EOExecutor
from eolearn.io import SentinelHubDemTask


In [ ]:
# This tutorial assumes all the patches are saved in current directory in folder Slovenia. You can change this here
patches_path = f'{os.path.abspath(os.getcwd())}/Slovenia'

## 1. Data sample construction
When training the classifier we don't want to include the pixels on the borders of parcels. These pixels are potential mixed-class instances that can have a negative effect on the learning process. So prior to sampling we will construct an timeless mask which excludes the edges. This is already done in an EOTask so we just need to call it.

Since we will be classificating crops we will calculate edges based on the NDVI metric and the green band.

In [ ]:
class AddNDVIGreenTask(EOTask):
    def __init__(nir=7,red=3,green=2):
        self.nir = nir
        self.red = red
        self.green = green
        
    def execute(self, eopatch):
        nir = eopatch.data['BANDS'][..., [self.nir]]
        red = eopatch.data['BANDS'][..., [self.red]]
        green = eopatch.data['BANDS'][..., [self.green]]

        eopatch.add_feature(FeatureType.DATA, 'GREEN', green)
        
        ndvi = np.clip((nir - red) / (nir + red + 0.000000001), -1, 1)
        eopatch.add_feature(FeatureType.DATA, 'NDVI', ndvi)
        
        return eopatch

base = AddNDVIGreenTask()
edges = EdgeExtractionTask(features={FeatureType.DATA: ['NDVI', 'GREEN']},
                          output_feature=(FeatureType.MASK_TIMELESS)

execution_args = []
for name,_ in os.walk(patches_path)
        execution_args.append({
            load: {'eopatch_folder': name},
            save: {'eopatch_folder': name}
        })
load = LoadTask(patches_path)

# save_patch_location = patches_path
save_patch_location = f'{os.path.abspath(os.getcwd())}/Slovenia2' # #### CHANGE

if not os.path.isdir(save_path_location):
    os.makedirs(save_path_location)
save = SaveTask(save_path_location, overwrite_permission=OverwritePermission.OVERWRITE_PATCH)

workflow = LinearWorkflow(load,
                         base,
                         edges,
                         save)

executor =  EOExecutor(workflow, execution_args)
executor.run(multiprocess=True)


In [ ]:
# Visualize the masks
images_path = 'Images'
if not os.path.isdir(images_path):
    os.makedirs(images_path)

for name,_ in os.walk(patches_path)
    eopatch = load(name,feaures=[(FeatureType.DATA,'BANDS'),(FeatureType.MASK_TIMELESS,'EDGE_INV')

In [ ]:
#Sampling

## Stream features calculation
From samples we will calculate the stream features.
We will also add height of the pixel as one of the features.


In [ ]:
dem = SentinelHubDemTask((FeatureType.DATA_TIMELESS, 'DEM'), size=(500, 505))